# DT MLOps-Zoomcamp 2024 - Homework #1
## Khalid AL Rahbi

In [12]:
pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 99.8 kB/s eta 0:00:00m eta 0:00:010:01:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 612.3 kB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [35]:
train_filename = './data/yellow_tripdata_2023-01.parquet'
validation_filename = "./data/yellow_tripdata_2023-02.parquet"
categorical_variables = ["PULocationID", "DOLocationID"]

def process_dataframe(filename: str) -> pd.DataFrame:
    data =  pd.read_parquet(filename)
    data["duration"] = ( data["tpep_dropoff_datetime"] - data["tpep_pickup_datetime"]) / pd.Timedelta('60s')
    data = data[ (data["duration"] >= 1) & (data["duration"] <= 60) ]
    data[categorical_variables] = data[categorical_variables].astype(str) 
    
    return data

### Q1. Downloading the data, Download the data for January and February 2023. Read the data for January. How many columns are there?



In [39]:
jan =  pd.read_parquet(train_filename)
len(jan.columns)

19

### Q2. Computing duration
### Now let's compute the duration variable. It should contain the duration of a ride in minutes.
### What's the standard deviation of the trips duration in January?

In [40]:
jan["duration"] = ( jan["tpep_dropoff_datetime"] - jan["tpep_pickup_datetime"]) / pd.Timedelta('60s')
round(jan["duration"].std(),2)

42.59

### Q3. Dropping outliers
### Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).
### What fraction of the records left after you dropped the outliers?

In [33]:
train_df = process_dataframe(filename=train_filename)

In [18]:
round((train_df.shape[0] / jan.shape[0]) * 100 , 2) 

98.12

In [19]:
del jan

### Q4. One-hot encoding
### Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.
### What's the dimensionality of this matrix (number of columns)?

In [20]:
train_dictionary = train_df[categorical_variables].to_dict(orient="records")

In [21]:
dv = DictVectorizer()
x_train = dv.fit_transform(train_dictionary)

In [22]:
x_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

### Q5. Training a model
### Now let's use the feature matrix from the previous step to train a model.
### Train a plain linear regression model with default parameters
### Calculate the RMSE of the model on the training data
### What's the RMSE on train?

In [23]:
y_train = train_df["duration"].values

In [24]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [25]:
predictions = lr.predict(x_train)

In [26]:
mean_squared_error(y_train, predictions, squared=False)

/home/khalid/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649261936488479

Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [41]:
validation_df = process_dataframe(filename=validation_filename)

validation_dictionary = validation_df[categorical_variables].to_dict(orient="records")
x_test = dv.transform(validation_dictionary)

In [42]:
y_test = validation_df["duration"].values

In [43]:
feb_predictions = lr.predict(x_test)
mean_squared_error(y_test, feb_predictions, squared=False)

/home/khalid/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.811818087547226